<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 20.60 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
%cd triviaqa

/content/triviaqa


In [3]:
!pip install -r requirements.txt

In [4]:
import subprocess
import os
import json
from tqdm import tqdm
import importlib
import argparse

In [5]:
env = os.environ.copy()
env["PYTHONPATH"] = "/content/triviaqa"

In [6]:
%cd ../model

/content/model


In [7]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: langchain-core
  

In [18]:
%cd ..

/content/model


In [23]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  return result.stdout

In [21]:
def get_answers(directory, questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    write_data(directory + "/predictions.json", answers)

In [11]:
def load_questions(path, split):
    create_split(path, split)

    with open(f"{path}/evaluation_dataset.json", "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(len(data["Data"])),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [12]:
def create_split(path, split):
  with open(f"{path}/original_dataset.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

  data["Data"] = data["Data"][:split]

  write_data(f"{path}/evaluation_dataset.json", data)

In [13]:
def load_model(path, model_name):
  print("Loading model...")
  module = importlib.import_module(path)
  model = getattr(module, model_name)
  return model()

In [14]:
def write_data(path, data):
  with open(path, "w", encoding="utf-8") as f:
    f.write(json.dumps(data))

In [15]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [20]:
if __name__ == "__main__":
    #args = get_args()
    directory = "/content/evaluation_data"
    split = 7900
    model_path = "model.rag_qa_embeddings"
    model_name = "rag_qa_embeddings"

    questions = load_questions(directory, split)
    model = load_model(model_path, model_name)
    print("Model initiated")
    print("Starting QA...")
    answers = get_answers(directory, questions, model)
    print("QA complete")
    print("Test results:")
    results = evaluate_triviaqa(directory + "/evaluation_dataset.json", directory + "/predictions.json")
    write_data(model_name + "_results.txt", results)
    print(results)


Loading Questions: 100%|██████████| 7900/7900 [00:00<00:00, 1043162.12it/s]


Loading model...


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Model initiated
Starting QA...


Retrieving Answers:  22%|██▏       | 1725/7900 [11:25<40:55,  2.52it/s]


KeyboardInterrupt: 